In [0]:
import numpy as np
from scipy.optimize import linprog

def nn2na(NN):
    # Get every location where exist an arc:
  arcs = np.argwhere(NN)
  NA = np.zeros([NN.shape[0], arcs.shape[0]]).astype(int)
  for i, arc in enumerate(arcs):
    NA[arc[0], i] = 1
    NA[arc[1], i] = -1

  arc_idx = [ (arc[0], arc[1]) for arc in arcs]
  return NA, arc_idx

# Shortest path Utils
def get_selected_arcs(arc_idxs, selected_arcs):
    arc = []
    for idx, i in enumerate(selected_arcs):
        if np.isclose(i, 1, rtol=1e-05, atol=1e-08, equal_nan=False): # Vecinity
            arc.append(arc_idxs[idx])
    return arc

def get_usage_string(arc_idxs, res_flow, capacity):
    return {arc: '%s/%s' % (flow, cap) for arc, flow, cap in zip(arc_idxs, res_flow, capacity)}

def get_min_cut(arc_idxs, np_res_flow, np_capacity):
    np_capacity = np.where(np_capacity == None, 999, np_capacity)

In [19]:
import numpy as np
from scipy.optimize import linprog

#arc_usage
#node_name = ['s', 's1', 's2' ,'s3', 's4', 's5', 's6', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 't1', 't2', 't3', 't4', 't5', 't6', 't']
arcs_full = np.array([
            'S-S1', 'S-S2', 'S-S3', 'S-S4', 'S-S5', 'S-S6',
            'S1-A2', 'S1-B4',
            'S2-A3', 'S2-B5',
            'S3-A4', 'S3-B6',
            'S4-A5',
            'S5-A6',
            'A1-B3', 'A1-T2', 
            'A2-B4', 'A2-T3', 
            'A3-B5', 'A3-T4', 
            'A4-B6', 'A4-T5', 
            'A5-T6',
            'B1-T2',
            'B2-T3',
            'B3-T4',
            'B4-T5',
            'B5-T6',
            'T1-T', 'T2-T', 'T3-T', 'T4-T', 'T5-T', 'T6-T',
            'T-S'
        ])

               #s s1 s2 s3 s4 s5 s6 a1 a2 a3 a4 a5 a6 b1 b2 b3 b4 b5 b6 t1 t2 t3 t4 t5 t6  t
NN = np.array([[0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],#s
               [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],#s1
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],#s2
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],#s3
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],#s4
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],#s5
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],#s6
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],#a1
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],#a2
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],#a3
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],#a4
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],#a5
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],#a6
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],#b1
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],#b2
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],#b3
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],#b4
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],#b5
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],#b6
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],#t1
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],#t2
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],#t3
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],#t4
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],#t5
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],#t6
               [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])#t

beq = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

C = np.zeros(len(np.argwhere(NN)))
C[-1] = -1

Aeq, arcs = nn2na(NN)
Aeq2, arc_idxs_list = nn2na(NN)

upper_b = [None, None, None, None, None, None, 5, 10, 5, 10, 5, 10, 5, 5, 6, 3, 6, 3, 6, 3, 6, 3, 6, 3, 3, 3, 3, 3, None, None, None, None, None, None, None]
bounds = tuple([(0, upper_b[arcs]) for arcs in range(0, Aeq.shape[1])])

res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')

print('Max flox: %s' % (res.fun * -1))
indexes = np.where(np.array(res.x) > 0.9)
#arc_names = find_arc_names(arc_idxs_list, node_name)
#arc_names
#for i in indexes:
#    print('%s: %s' % (arcs_full[i], res.x[i]))
print("Quantities: ")
for i in range(len(res.x)):
    print(arcs[i], ":", res.x[i].astype(int))


Max flox: 20.0
Quantities: 
(0, 1) : 6
(0, 2) : 6
(0, 3) : 3
(0, 4) : 5
(0, 5) : 0
(0, 6) : 0
(1, 8) : 5
(1, 16) : 1
(2, 9) : 5
(2, 17) : 1
(3, 10) : 3
(3, 18) : 0
(4, 11) : 5
(5, 12) : 0
(7, 15) : 0
(7, 20) : 0
(8, 16) : 2
(8, 21) : 3
(9, 17) : 2
(9, 22) : 3
(10, 18) : 0
(10, 23) : 3
(11, 24) : 5
(13, 20) : 0
(14, 21) : 0
(15, 22) : 0
(16, 23) : 3
(17, 24) : 3
(19, 25) : 0
(20, 25) : 0
(21, 25) : 3
(22, 25) : 3
(23, 25) : 6
(24, 25) : 8
(25, 0) : 20


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
